## This code diagonalizes the model Hamiltonians for a H2 dimer where one monomer is active (can relax) and the other is a frozen environment (Triplet or Singlet)


Updated: Pablo ROSEIRO - 25.06.2023

Last code update: Pablo ROSEIRO - 22.02.2022

More details are contained directly into the functions used in this work.

This is a system of 4e- in 4 MOs.

The Singlet space comprises three states, whereas the Triplet space comprises three DEGENERATED states.

A Singlet environment, S_b, is built upon the contraction of the three singlet states.

The Triplet environment is denoted T_b.

S_a and T_a are states of the active moiety.

It is important to remind the following:

S_a x S_b -> 3x3

S_a x T_b -> 3x3

Ta x Sb -> 1x1

Ta x Tb -> 3 matrices of 1x1

In [1]:
from __future__ import print_function

from scipy.linalg import eigh
import numpy

from IPython.display import display

import pandas as pd

import os

This function develops the MOs integrals as linear combinations of AOs integrals.

g, u, j and v correspond, respectively, to g, u, g' and u'.

In [2]:
def integrals(S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6):
    """
    Below are the analytical formula of MOs integrals developed in the basis of AOs integrals.
    This eases automatization and Potential Energy Curves construction.
    """
    g_h_g = 1/(2*(1+S)) * (2*eps + 2*t)
    u_h_u = 1/(2*(1-S)) * (2*eps - 2*t)
    
    g_h_j = 1/(2*(1+S)) * (2*t_i + 2*t_i2)
    u_h_v = 1/(2*(1-S)) * (2*t_i - 2*t_i2)
    
    S1 = 1/(2*(1+S)) * (2*S_i + 2*S_i2)
    S3 = 1/(2*(1-S)) * (2*S_i - 2*S_i2)
    
    gggg = 1/(4*(1+S)**2) * (2*U + 2*J + 8*lambda1 + 4*K)
    ggjj = 1/(4*(1+S)**2) * (2*Jbis + 2*J1 + 8*lambda2 + 4*omega1)
    gjjg = 1/(4*(1+S)**2) * (2*Kbis + 2*theta1 + 8*theta2 + 2*omega2 + 2*omega3)
    uuuu = 1/(4*(1-S)**2) * (2*U + 2*J - 8*lambda1 + 4*K)
    gguu = 1/(4*(1-S**2)) * (2*U + 2*J - 4*K)
    guug = 1/(4*(1-S**2)) * (2*U - 2*J)
    ggvv = 1/(4*(1-S**2)) * (2*Jbis + 2*J1 - 4*omega1)
    gvvg = 1/(4*(1-S**2)) * (2*Kbis - 2*theta1 + 2*omega2 - 2*omega3)
    uuvv = 1/(4*(1-S)**2) * (2*Jbis + 2*J1 - 8*lambda2 + 4*omega1)
    uvvu = 1/(4*(1-S)**2) * (2*Kbis + 2*theta1 - 8*theta2 + 2*omega2 + 2*omega3)
    gujv = 1/(4*(1-S**2)) * (2*Jbis - 2*J1)
    gvju = 1/(4*(1-S**2)) * (2*Kbis - 2*theta1 - 2*omega2 + 2*omega3)
    gjuv = 1/(4*(1-S**2)) * (2*Kbis + 2*theta1 - 2*omega2 - 2*omega3)
    gggj = 1/(4*(1+S)**2) * (2*gamma1 + 2*gamma2 + 2*gamma3 + 2*gamma4 + 4*gamma5 + 4*gamma6)
    gjuu = 1/(4*(1-S**2)) * (2*gamma1 + 2*gamma2 + 2*gamma3 + 2*gamma4 - 4*gamma5 - 4*gamma6)
    guuj = 1/(4*(1-S**2)) * (2*gamma1 - 2*gamma2 + 2*gamma3 - 2*gamma4)
    uuuv = 1/(4*(1-S)**2) * (2*gamma1 + 2*gamma2 - 2*gamma3 - 2*gamma4 - 4*gamma5 + 4*gamma6)
    gguv = 1/(4*(1-S**2)) * (2*gamma1 + 2*gamma2 - 2*gamma3 - 2*gamma4 + 4*gamma5 - 4*gamma6)
    guvg = 1/(4*(1-S**2)) * (2*gamma1 - 2*gamma2 - 2*gamma3 + 2*gamma4)
    
    
    return g_h_g, u_h_u, g_h_j, u_h_v, S, S1, S3, gggg, ggjj, gjjg, uuuu, gguu, guug, ggvv, gvvg, uuvv, uvvu, gujv, gvju, gjuv, gggj, gjuu, guuj, uuuv, gguv, guvg


The two functions below are the model Hamiltonians (of a Singlet or Triplet environment) in the determinant basis.

In [3]:
def Mat_EnvironSinglet(L,l, coeff_gs, coeff_es, coeff_os, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6):
    """
    Scheme :
        b|     |b'
         |     |
        a|     |a'
        
    This model Hamiltonian studies the influence of a contracted Singlet environment on an active H2 monomer.
    The Singlet contracted environment is normalized:
    S_E = coeff_gs * |g_E -g_E| + coeff_es * |u_E -u_E| + coeff_os * [ 1/sqrt(2) (|g_E -u_E| + |u_E -g_E|) ]
    Regarding the active moiety:
    a* = |g_A -g_A|
    b* = |u_A -u_A|
    c* = |g_A -u_A|
    d* = |u_A -g_A|
    """
    ### Normalization
    norm = coeff_gs ** 2 + coeff_es ** 2 + coeff_os ** 2

    E1 = coeff_gs * numpy.sqrt(1 / norm)
    E2 = coeff_es * numpy.sqrt(1 / norm)
    E3 = coeff_os * numpy.sqrt(1 / norm)
    

    ### Extract the values of MOs integrals for a given geometry
    g_h_g, u_h_u, g_h_j, u_h_v, S, S1, S3, gggg, ggjj, gjjg, uuuu, gguu, guug, ggvv, gvvg, uuvv, uvvu, gujv, gvju, gjuv, gggj, gjuu, guuj, uuuv, gguv, guvg = integrals(S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)

    
    ### MATRIX ELEMENTS
    ## GS singlet environment: |g_E -g_E|
    # Diagonal elements
    a1a1 = 4*S1**2*gjjg - 8*S1*gggj + 4*g_h_g*(1 - S1**2) + 4*g_h_j*(S1**3 - S1) + 2*(1 - S1**2)*(ggjj - gjjg) + gggg + 2*ggjj + gggg
    b1b1 = 2*g_h_g + 2*u_h_u + gggg + 4*ggvv + uuuu - 2*gvvg
    c1c1 = -2*S1*g_h_j - 2*S1*gjuu - 2*S1*gggj + g_h_g*(1 - S1**2) + 2*g_h_g + u_h_u*(1 - S1**2) + (1 - S1**2)*(ggvv - gvvg) + 2*ggjj + gguu + gggg + ggvv - gjjg
    d1d1 = c1c1
    # Extra diagonal terms
    a1b1 = S1**2*gvvg - 2*S1*guuj + guug 
    a1c1 = 0
    a1d1 = 0
    b1c1 = 0
    b1d1 = 0
    c1d1 = a1b1
    
    ## ES singlet environment: |u_E -u_E|
    # Diagonal elements
    a2a2 = b1b1
    b2b2 = 4*S3**2*uvvu - 8*S3*uuuv + 4*u_h_u*(1 - S3**2) + 4*u_h_v*(S3**3 - S3) + 2*(1 - S3**2)*(uuvv - uvvu) + uuuu + 2*uuvv + uuuu
    c2c2 = -2*S3*u_h_v - 2*S3*gguv - 2*S3*uuuv + g_h_g*(1 - S3**2) + u_h_u*(1 - S3**2) + 2*u_h_u + (1 - S3**2)*(ggvv - gvvg) + gguu + ggvv + 2*uuvv + uuuu - uvvu
    d2d2 = c2c2
    # Extra diagonal terms
    a2b2 = S3**2*gvvg - 2*S3*guvg + guug
    a2c2 = 0
    a2d2 = 0
    b2c2 = 0
    b2d2 = 0
    c2d2 = a2b2

    ## determinant of OS environment: |g -u|
    # Diagonal elements
    a3a3 = -2*S1*g_h_j - 2*S1*gggj - 2*S1*gjuu + g_h_g*(1 - S1**2) + 2*g_h_g + u_h_u*(1 - S1**2) + (1 - S1**2)*(ggvv - gvvg) + gggg + 2*ggjj + ggvv + gguu - gjjg
    b3b3 = -2*S3*u_h_v - 2*S3*gguv - 2*S3*uuuv + g_h_g*(1 - S3**2) + u_h_u*(1 - S3**2) + 2*u_h_u + (1 - S3**2)*(ggvv - gvvg) + gguu + ggvv + uuuu + 2*uuvv - uvvu
    c3c3 = 4*S1*S3*gjuv - 4*S1*gjuu - 4*S3*gguv + 2*g_h_g*(1 - S3**2) + 2*g_h_j*(S1*S3**2 - S1) + 2*u_h_u*(1 - S1**2) + 2*u_h_v*(S1**2*S3 - S3) + (1 - S1**2)*(uuvv - uvvu) + (1 - S3**2)*(ggjj - gjjg) + gguu + ggvv + gguu + ggvv
    d3d3 = 2*g_h_g + 2*u_h_u + ggjj + ggvv + 2*gguu + ggvv + uuvv - 2*gvvg
    # Extra diagonal terms
    a3b3 = S1*S3*gvju - S1*guuj - S3*guvg + guug
    a3c3 = 0
    a3d3 = 0
    b3c3 = 0
    b3d3 = 0
    c3d3 = a3b3
    
    ## determinant of OS environment: |u -g|
    # Diagonal elements
    a4a4 = a3a3
    b4b4 = b3b3
    c4c4 = d3d3
    d4d4 = c3c3
    # Extra diagonal terms
    a4b4 = a3b3
    a4c4 = 0
    a4d4 = 0
    b4c4 = 0
    b4d4 = 0
    c4d4 = a4b4
    
    ## EXTRA COUPLINGS
    # The next part was computed automatically using the script (Symbolic_CALC_matrixElements.ipynb)
    a1a2 = a3a4 = S1**2*gvvg - 2*S1*guuj + guug
    d1d2 = c1c2 = c3c4 = d3d4 = S1*S3*gvju - S1*guuj - S3*guvg + guug
    b1b2 = b3b4 = S3**2*gvvg - 2*S3*guvg + guug
    a1b2 = a1c2 = a1d2 = a1b3 = a1b4 = 0
    b1a2 = 2*S1**2*S3*u_h_v + S1**2*uvvu + 2*S1*S3**2*g_h_j - 2*S1*S3*(gujv - gjuv) + 2*S1*S3*gjuv + S3**2*gjjg
    b1c2 = b1d2 = b1a3 = c1d3 = 0
    c1d2 = -S1*S3*(gujv - gvju)
    c1b2 = b1a4 = c1a2 = a2b4 = a2b3 = d1c4 = d1c3 = 0
    d1c2 = c1d2
    d1b2 = d1a2 = c1d4 = c2d4 = c2d3 = b2a4 = b2a3 = d2c3 = d2c4 = 0
    a3b4 = c1d2
    a3c4 = a3d4 = 0 
    b3a4 = c1d2
    b3c4 = b3d4 = c3a4 = c3b4 = c3d4 = d3a4 = d3b4 = 0
    d3c4 = 2*S1**2*S3*u_h_v + S1**2*uvvu + 2*S1*S3**2*g_h_j - 2*S1*S3*gujv + 4*S1*S3*gjuv + S3**2*gjjg #4 MOs of diff ?
    a1c3 = (1-S1**2)*(gujv - gvju) 
    b2c3 = (1-S3**2)*(gujv - gvju)
    d2a3 = c2a4 = gujv - gvju
    b2d4 = b2c3
    d1b3 = c1b4 = d2a3
    a1d4 = a1c3 
    a1d3 = d1a3 = c1a4 = a1c4 = S1**2*gvju - 2*S1*guvg + gujv
    b2c4 = b2d3 = d2b3 = c2b4 = S3**2*gvju - 2*S3*guuj + gujv
    d2a4 = c2a3 = d1b4 = b1c3 = a2d4 = a2c3 =  c1b3 = b1d4 = S1*S3*gvvg - S1*guvg - S3*guuj + gujv
    b1d3 = b1c4 = -S1*S3*g_h_g - S1*S3*u_h_u - S1*S3*(ggvv - gvvg) - S1*u_h_v - S1*uuuv - S1*gguv - S3*g_h_j - S3*gggj - S3*gjuu + gujv - gjuv
    a2c4 = a2d3 = -S1*S3*g_h_g - S1*S3*u_h_u - S1*S3*(ggvv - gvvg) - S1*u_h_v - S1*gguv - S1*uuuv - S3*g_h_j - S3*gggj - S3*gjuu - gjuv + gujv
    d2b4 = c2b3 = 2*S1*S3**2*u_h_v - 2*S1*S3*u_h_u - S1*S3*(uuvv - uvvu) + 2*S1*S3*uvvu - S1*uuuv - S1*uuuv + 2*S3**2*gjuv - 2*S3*gjuu + g_h_j*(S3**3 - S3) + u_h_v*(S1*S3**2 - S1) + (1 - S3**2)*(gujv - gjuv)
    d1a4 = c1a3 = 2*S1**2*S3*g_h_j + 2*S1**2*gjuv - 2*S1*S3*g_h_g - S1*S3*(ggjj - gjjg) + 2*S1*S3*gjjg - 2*S1*gguv - 2*S3*gggj + g_h_j*(S1**2*S3 - S3) + u_h_v*(S1**3 - S1) + (1 - S1**2)*(gujv - gjuv)
    a1a3 = a1a4 = b1b3 = b1b4 = c1c3 = c1c4 = d1d3 = d1d4 = a2a3 = a2a4 = b2b3 = b2b4 = c2c3 = c2c4 = d2d3 = d2d4 = 0
  

    ### CONTRACTION OF MATRIX ELEMENTS
    ## Nuclear effects terms (diagonal terms)
    DIAG = 2/L + 2/numpy.sqrt(L**2 + l**2) + 2/l
    
    ## 3x3 SINGLET Matrix (Three Singlet states)
    # Diagonal terms
    aa = DIAG + E1**2 * a1a1 + E2 **2 * a2a2 + 1/2 * E3 **2 * (a3a3 + a4a4 + 2*a3a4) \
            + 2*E1*E2*a1a2 + numpy.sqrt(2)*E1*E3*(a1a3 + a1a4) + numpy.sqrt(2)*E2*E3*(a2a3 + a2a4)
    
    bb = DIAG + E1**2 * b1b1 + E2 **2 * b2b2 + 1/2 * E3 **2 * (b3b3 + b4b4 + 2*b3b4) \
            + 2*E1*E2*b1b2 + numpy.sqrt(2)*E1*E3*(b1b3 + b1b4) + numpy.sqrt(2)*E2*E3*(b2b3 + b2b4)
    
    cc = DIAG + 1/2 * (E1**2 * (c1c1 + d1d1 + 2*c1d1) + E2**2 * (c2c2 + d2d2 + 2*c2d2) \
            + 1/2 * E3**2 * (c3c3 + d3d3 + 2*c3d3 + c4c4 + d4d4 + 2*c4d4 \
            + 2*(c3c4 + d3d4) + 2*(c3d4 + d3c4)) + 2*E1*E2*(c1c2 + d1d2 + c1d2 + d1c2) \
            + numpy.sqrt(2)*E1*E3*(c1c3 + c1d3 + d1c3 + d1d3 + c1c4 + c1d4 + d1c4 + d1d4) \
            + numpy.sqrt(2)*E2*E3*(c2c3 + c2d3 + d2c3 + d2d3 + c2c4 + c2d4 + d2c4 + d2d4))
    
    # Extra diagonal terms
    ab = E1**2 * a1b1 + E2 **2 * a2b2 + 1/2 * E3 **2 * (a3b3 + a4b4 + a3b4 + b3a4) \
            + E1*E2*(a1b2 + b1a2) + 1/numpy.sqrt(2) *E1*E3*(a1b3 + b1a3 + a1b4 + b1a4) \
            + 1/numpy.sqrt(2)*E2*E3*(a2b3 + b2a3 + a2b4 + b2a4)
    
    ac = 1/numpy.sqrt(2)*(E1**2 * (a1c1 + a1d1) + E2**2 * (a2c2 + a2d2) \
            + 1/2 * E3**2 *(a3c3 + a4c4 + a3c4 + c3a4 + a3d3 + a4d4 + a3d4 + d3a4) \
            + E1*E2 * (a1c2 + c1a2 + a1d2 + d1a2) \
            + 1/numpy.sqrt(2)*E1*E3 * (a1c3 + a1c4 + c1a3 + c1a4 + a1d3 + a1d4 + d1a3 + d1a4) \
            + 1/numpy.sqrt(2)*E2*E3 * (a2c3 + a2c4 + c2a3 + c2a4 + a2d3 + a2d4 + d2a3 + d2a4))
    
    bc = 1/numpy.sqrt(2)*(E1**2 * (b1c1 + b1d1) + E2**2 * (b2c2 + b2d2) \
            + 1/2 * E3**2 *(b3c3 + b4c4 + b3c4 + c3b4 + b3d3 + b4d4 + b3d4 + d3b4) \
            + E1*E2 * (b1c2 + c1b2 + b1d2 + d1b2) \
            + 1/numpy.sqrt(2)*E1*E3 * (b1c3 + b1c4 + c1b3 + c1b4 + b1d3 + b1d4 + d1b3 + d1b4) \
            + 1/numpy.sqrt(2)*E2*E3 * (b2c3 + b2c4 + c2b3 + c2b4 + b2d3 + b2d4 + d2b3 + d2b4))
    
    ## 1x1 TRIPLET Matrix (One Triplet state)
    dd = DIAG + 1/2 * (E1**2 * (c1c1 + d1d1 - 2*c1d1) + E2**2 * (c2c2 + d2d2 - 2*c2d2) \
            + 1/2 * E3**2 * (c3c3 + c4c4 + 2*c3c4 + d3d3 + d4d4 + 2*d3d4 \
            - 2*(c3d3 + c4d4) - 2*(c3d4 + d3c4)) + 2*E1*E2*(c1c2 + d1d2 - c1d2 - d1c2) \
            + numpy.sqrt(2)*E1*E3*(c1c3 - c1d3 - d1c3 + d1d3 + c1c4 - c1d4 - d1c4 + d1d4) \
            + numpy.sqrt(2)*E2*E3*(c2c3 - c2d3 - d2c3 + d2d3 + c2c4 - c2d4 - d2c4 + d2d4))
    
    
    ### CONSTRUCTION OF THE FINAL MATRIX for a Singlet environment
    H = [[aa, ab, ac],[ab, bb, bc],[ac, bc, cc]]
    return numpy.array(H), dd

In [4]:
def Mat_EnvironTriplet(L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6):
    """
    Scheme :
        b|     |b'
         |     |
        a|     |a'
        
    This model Hamiltonian studies the influence of a Triplet environment on an active H2 monomer.
    To incorporate M_S = 0 determinants resulting of the tensor product of m_s =! 0 local determinants (e.g. |g_A u_A -g_E -u_E),
    one need to develop matrix elements resulting from the tensor product with the m_s = +-1 on the Triplet environment.
    Regarding the active moiety:
    a* = |g_A  -g_A|
    b* = |u_A  -u_A|
    c* = |g_A  -u_A|
    d* = |u_A  -g_A|
    e* = |g_A   u_A|
    f* = |-g_A -u_A|
    """
    ### Extract the values of MOs integrals for a given geometry
    g_h_g, u_h_u, g_h_j, u_h_v, S, S1, S3, gggg, ggjj, gjjg, uuuu, gguu, guug, ggvv, gvvg, uuvv, uvvu, gujv, gvju, gjuv, gggj, gjuu, guuj, uuuv, gguv, guvg = integrals(S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)

    
    ### MATRIX ELEMENTS
    ## Triplet environment m_s = +1: |g_E u_E|
    # Diagonal elements
    a1a1 = -2*S1*g_h_j - 2*S1*(gjuu - guuj) - 2*S1*gggj + g_h_g*(1 - S1**2) + 2*g_h_g + u_h_u*(1 - S1**2) + (1 - S1**2)*ggvv + gggg + 2*ggjj + ggvv + gguu - gjjg - gvvg - guug
    b1b1 = -2*S3*u_h_v + 2*S3*(guvg - gguv) - 2*S3*uuuv + g_h_g*(1 - S3**2) + u_h_u*(1 - S3**2) + 2*u_h_u + (1 - S3**2)*ggvv + gguu + ggvv + uuuu + 2*uuvv - gvvg - guug - uvvu
    c1c1 = -2*S1*g_h_j - 2*S1*(gjuu - guuj) - 2*S1*gjuu + 2*g_h_g + 2*u_h_u*(1 - S1**2) + (1 - S1**2)*uuvv + ggjj + gguu + ggvv + gguu + ggvv - gjjg - gvvg - guug
    d1d1 = -2*S3*u_h_v - 2*S3*(gguv - guvg) - 2*S3*gguv + 2*u_h_u + 2*g_h_g*(1 - S3**2) + (1 - S3**2)*ggjj + ggvv + gguu + gguu + ggvv + uuvv - gvvg - guug - uvvu
    # Extra diagonal terms
    a1b1 = S1*S3*gujv - S1*guuj - S3*guvg + guug
    a1c1 = 0
    a1d1 = 0
    b1c1 = 0
    b1d1 = 0
    c1d1 = a1b1
    # m_s = +2 and m_s = 0 terms
    e1e1 = 2*S1*S3*(gjuv - gujv) + 2*S1*S3*(gjuv - gvju) + 4*S1*(guuj - gjuu) - 4*S3*(gguv - guvg) + 2*g_h_g*(1 - S3**2) + 2*g_h_j*(S1*S3**2 - S1) + 2*u_h_u*(1 - S1**2) + 2*u_h_v*(S1**2*S3 - S3) + (1 - S1**2)*(uuvv - uvvu) + (1 - S3**2)*(ggjj - gjjg) + 2*gguu + 2*ggvv - 2*guug - 2*gvvg  
    f1f1 = 2*g_h_g + 2*u_h_u + ggjj + 2*gguu + 2*ggvv + uuvv - 2*guug
    
    ## Triplet environment m_s = -1: |-g_E -u_E|
    # Diagonal elements
    a2a2 = a1a1
    b2b2 = b1b1
    c2c2 = d1d1
    d2d2 = c1c1
    # Extra diagonal terms
    a2b2 = a1b1
    a2c2 = a1d1
    a2d2 = a1c1
    b2c2 = b1d1
    b2d2 = b1c1
    c2d2 = a2b2
    # m_s = 0 and m_s = -2 terms
    e2e2 = f1f1
    f2f2 = e1e1
    
    ## determinant of m_s = 0 environment: |g -u|
    # Diagonal elements
    a3a3 = -2*S1*g_h_j - 2*S1*gggj - 2*S1*gjuu + g_h_g*(1 - S1**2) + 2*g_h_g + u_h_u*(1 - S1**2) + (1 - S1**2)*(ggvv - gvvg) + gggg + 2*ggjj + ggvv + gguu - gjjg
    b3b3 = -2*S3*u_h_v - 2*S3*gguv - 2*S3*uuuv + g_h_g*(1 - S3**2) + u_h_u*(1 - S3**2) + 2*u_h_u + (1 - S3**2)*(ggvv - gvvg) + gguu + ggvv + uuuu + 2*uuvv - uvvu
    c3c3 = 4*S1*S3*gjuv - 4*S1*gjuu - 4*S3*gguv + 2*g_h_g*(1 - S3**2) + 2*g_h_j*(S1*S3**2 - S1) + 2*u_h_u*(1 - S1**2) + 2*u_h_v*(S1**2*S3 - S3) + (1 - S1**2)*(uuvv - uvvu) + (1 - S3**2)*(ggjj - gjjg) + gguu + ggvv + gguu + ggvv
    d3d3 = 2*g_h_g + 2*u_h_u + ggjj + ggvv + 2*gguu + ggvv + uuvv - 2*gvvg
    # Extra diagonal terms
    a3b3 = S1*S3*gvju - S1*guuj - S3*guvg + guug
    a3c3 = 0
    a3d3 = 0
    b3c3 = 0
    b3d3 = 0
    c3d3 = a3b3
    # m_s = +1 and m_s = -1 terms
    e3e3 = -2*S1*g_h_j + 2*S1*(guuj - gjuu) - 2*S1*gjuu + 2*g_h_g + 2*u_h_u*(1 - S1**2) + (1 - S1**2)*uuvv + ggjj + 2*gguu + 2*ggvv - gjjg - guug - gvvg
    f3f3 = -2*S3*u_h_v - 2*S3*(gguv - guvg) - 2*S3*gguv + 2*g_h_g*(1 - S3**2) + 2*u_h_u + (1 - S3**2)*ggjj + 2*gguu + 2*ggvv + uuvv - guug - gvvg - uvvu
    
    ## determinant of m_s = 0 environment: |u -g|
    # Diagonal elements
    a4a4 = a3a3
    b4b4 = b3b3
    c4c4 = d3d3
    d4d4 = c3c3
    # Extra diagonal terms
    a4b4 = a3b3
    a4c4 = 0
    a4d4 = 0
    b4c4 = 0
    b4d4 = 0
    c4d4 = a4b4
    # m_s = +1 and m_s = -1 terms
    e4e4 = f3f3
    f4f4 = e3e3
    
    ## EXTRA COUPLINGS
    # The next part was computed automatically using the script (Symbolic_CALC_matrixElements.ipynb)
    a3a4 = S1**2*gvvg - 2*S1*guuj + guug
    b3b4 = S3**2*gvvg - 2*S3*guvg + guug
    c3c4 = d3d4 = S1*S3*gvju - S1*guuj - S3*guvg + guug
    a3b4 = -S1*S3*(gujv - gvju)
    d3c4 = 2*S1**2*S3*u_h_v + S1**2*uvvu + 2*S1*S3**2*g_h_j - 2*S1*S3*gujv + 4*S1*S3*gjuv + S3**2*gjjg
    a1a2 = d1d2 = c1c2 = b1b2 = 0
    a1b2 = a1c2 = a1d2 = a1b3 = a1b4 = 0
    b1a2 = 0
    b1c2 = b1d2 = b1a3 = c1d3 = 0
    c1d2 = 0
    c1b2 = b1a4 = c1a2 = a2b4 = a2b3 = d1c4 = d1c3 = 0
    d1c2 = 0
    d1b2 = d1a2 = c1d4 = c2d4 = c2d3 = b2a4 = b2a3 = d2c3 = d2c4 = 0
    a3c4 = a3d4 = 0 
    b3a4 = a3b4
    b3c4 = b3d4 = c3a4 = c3b4 = c3d4 = d3a4 = d3b4 = 0
    a1c3 = 0
    b2c3 = 0
    d2a3 = c2a4 = 0
    b2d4 = 0
    d1b3 = c1b4 = 0
    a1d4 = 0 
    a1d3 = b2c4 = b2d3 = d2a4 = c2a3 = d1b4 = a1c4 = b1c3 = a2d4 = d2b3 = c2b4 = a2c3 = d1a3 = c1a4 = c1b3 = b1d4 = 0
    b1d3 = a2c4 = a2d3 = 0
    d2b4 = c2b3 = 0
    d1a4 = c1a3 = 0
    b1c4 = 0
    a1a3 = a1a4 = 0
    b1b3 = b1b4 = 0
    c1c3 = 0
    c1c4 = d1d3 = 0
    d1d4 = 0
    a2a3 = a2a4 = 0
    b2b3 = b2b4 = 0
    c2c3 = 0
    c2c4 = d2d3 = 0
    d2d4 = 0
    #
    f1e2 = 2*S1**2*S3*u_h_v + S1**2*uvvu + 2*S1*S3**2*g_h_j + 2*S1*S3*(2*gjuv - gvju) + S3**2*gjjg
    e3e4 = f3f4 = S1*S3*gujv - S1*guuj - S3*guvg + guug
    e1f1 = e1e2 = e1f2 = e1e3 = e1f3 = e1e4 = e1f4 = 0
    f1f2 = f1e3 = f1f3 = f1e4 = f1f4 = e2f2 = e2e3 = e2f3 = 0
    e2e4 = e2f4 = f2e3 = f2f3 = f2e4 = f2f4 = e3f3 = e3f4 = 0
    f3e4 = e4f4 = 0
    #
    f1c3 = -S1*S3*gujv + S1*guuj + S3*guvg - gvvg
    f1d3 = 2*S3**2*g_h_g + S3**2*ggjj + 2*S3*u_h_v + S3*(gguv - guvg) - S3*(guvg - 3*gguv) + uvvu
    f1c4 = 2*S1**2*u_h_u + S1**2*uuvv + 2*S1*g_h_j - S1*(guuj - gjuu) + S1*(gjuu - guuj) + S1*gjuu + S1*gjuu + gjjg
    e3c1 = 2*S1*S3**2*g_h_j + S1*S3*(2*gjuv - gvju) - S1*S3*(gujv - 2*gjuv) - 2*S3**2*g_h_g - S3**2*(ggjj - gjjg) - 2*S3*(gguv - guvg) - 2*S3*gguv + 2*u_h_v*(S1**2*S3 - S3) - (1 - S1**2)*uvvu
    e3d1 = S1*S3*gvju - S1*guuj - S3*guvg + gvvg
    f3c2 = 2*S1**2*S3*u_h_v - 2*S1**2*u_h_u - S1**2*(uuvv - uvvu) - S1*S3*(gujv - gjuv) + S1*S3*(gjuv - gvju) + 2*S1*S3*gjuv + S1*(guuj - gjuu) - S1*(gjuu - guuj) - 2*S1*gjuu + 2*g_h_j*(S1*S3**2 - S1) - (1 - S3**2)*gjjg
    f3d2 = e3d1
    e2c4 = f1d3
    e2d4 = f1c3
    e2c3 = f1c3 
    e2d3 = f1c4
    f1d4 = f1c3
    e4c1 = e3d1
    e4d1 = f3c2
    f4c2 = e3d1
    f4d2 = e3c1
    e1a1 = e1b1 = e1c1 = e1d1 = e1a2 = e1b2 = e1c2 = e1d2 = e1a3 = e1b3 = e1c3 = e1d3 = e1a4 = e1b4 = e1c4 = e1d4 = 0
    f1a1 = f1b1 = f1c1 = f1d1 = f1a2 = f1b2 = f1c2 = f1d2 = f1a3 = f1b3 = 0 
    f1a4 = f1b4 = 0 
    e2a1 = e2b1 = e2c1 = e2d1 = e2a2 = e2b2 = e2c2 = e2d2 = e2a3 = e2b3 = 0 
    e2a4 = e2b4 = 0
    f2a1 = f2b1 = f2c1 = f2d1 = f2a2 = f2b2 = f2c2 = f2d2 = f2a3 = f2b3 = f2c3 = f2d3 = f2a4 = f2b4 = f2c4 = f2d4 = e3a1 = e3b1 = 0 
    e3a2 = e3b2 = e3c2 = e3d2 = e3a3 = e3b3 = e3c3 = e3d3 = e3a4 = e3b4 = e3c4 = e3d4 = f3a1 = f3b1 = f3c1 = f3d1 = f3a2 = f3b2 = 0 
    f3a3 = f3b3 = f3c3 = f3d3 = f3a4 = f3b4 = f3c4 = f3d4 = e4a1 = e4b1 = 0 
    e4a2 = e4b2 = e4c2 = e4d2 = e4a3 = e4b3 = e4c3 = e4d3 = e4a4 = e4b4 = e4c4 = e4d4 = f4a1 = f4b1 = f4c1 = f4d1 = f4a2 = f4b2 = 0
    f4a3 = f4b3 = f4c3 = f4d3 = f4a4 = f4b4 = f4c4 = f4d4 = 0
  
    ### MATRIX ELEMENTS
    # 1) m_s = -1, 0 and +1 are degenerated, thus one can consider an environment of solely one of environment determinant 
    # for the active Singlet states. 
    # 2) This is not the case when coupling a Triplet active state with a Triplet environment, giving rise to a Singlet, a Triplet and a Quintet states.
    # To verify the assumption (1), one may uncomment the further commented lines to satisfy its curiosity.

    ## Normalize given coefficients
#     coeff_plus = coeff_minus = coeff_0 = 1
#     norm = coeff_plus ** 2 + coeff_minus ** 2 + coeff_0 ** 2
    
#     E1 = coeff_plus * numpy.sqrt(1 / norm)
#     E2 = coeff_minus * numpy.sqrt(1 / norm)
#     E3 = coeff_0 * numpy.sqrt(1 / norm)

    ## Nuclear effects terms (diagonal terms)
    DIAG = 2/L + 2/numpy.sqrt(L**2 + l**2) + 2/l
    
    ## 4x4 TRIPLET Matrix (three local singlet states and one local triplet on the active part)
    # Diagonal terms
    aa = DIAG + a1a1 
#     aa = DIAG + E1**2 * a1a1 + E2 **2 * a2a2 + 1/2 * E3 **2 * (a3a3 + a4a4 - 2*a3a4) \
#          + 2*E1*E2*a1a2 + numpy.sqrt(2)*E1*E3*(a1a3 - a1a4) + numpy.sqrt(2)*E2*E3*(a2a3 - a2a4)

    bb = DIAG + b1b1
#     bb = DIAG + E1**2 * b1b1 + E2 **2 * b2b2 + 1/2 * E3 **2 * (b3b3 + b4b4 - 2*b3b4) \
#          + 2*E1*E2*b1b2 + numpy.sqrt(2)*E1*E3*(b1b3 - b1b4) + numpy.sqrt(2)*E2*E3*(b2b3 - b2b4)

    cc = DIAG + 1/2 * (c1c1 + d1d1 + 2*c1d1)
#     cc = DIAG + 1/2 * (E1**2 * (c1c1 + d1d1 + 2*c1d1) + E2**2 * (c2c2 + d2d2 + 2*c2d2) \
#          + 1/2 * E3**2 * (c3c3 + d3d3 + 2*c3d3 + c4c4 + d4d4 + 2*c4d4 \
#          - 2*(c3c4 + d3d4) - 2*(c3d4 + d3c4)) + 2*E1*E2*(c1c2 + d1d2 + c1d2 + d1c2) \
#          + numpy.sqrt(2)*E1*E3*(c1c3 + c1d3 + d1c3 + d1d3 - c1c4 - c1d4 - d1c4 - d1d4) \
#          + numpy.sqrt(2)*E2*E3*(c2c3 + c2d3 + d2c3 + d2d3 - c2c4 - c2d4 - d2c4 - d2d4))
    
    triptrip = DIAG + 1/2 * (e2e2 + f1f1 - 2*f1e2)

    # Extra diagonal terms
    ab = a1b1 
#     ab =  E1**2 * a1b1 + E2 **2 * a2b2 + 1/2 * E3 **2 * (a3b3 + a4b4 - a3b4 - b3a4) \
#           + E1*E2*(a1b2 + b1a2) + 1/numpy.sqrt(2) *E1*E3*(a1b3 + b1a3 - a1b4 - b1a4) \
#           + 1/numpy.sqrt(2)*E2*E3*(a2b3 + b2a3 - a2b4 - b2a4)
    
    ac = 1/numpy.sqrt(2)*(a1c1 + a1d1)
#     ac =  1/numpy.sqrt(2)*(E1**2 * (a1c1 + a1d1) + E2**2 * (a2c2 + a2d2) \
#           + 1/2 * E3**2 *(a3c3 + a4c4 - a3c4 - c3a4 + a3d3 + a4d4 - a3d4 - d3a4) \
#           + E1*E2 * (a1c2 + c1a2 + a1d2 + d1a2) \
#           + 1/numpy.sqrt(2)*E1*E3 * (a1c3 - a1c4 + c1a3 - c1a4 + a1d3 - a1d4 + d1a3 - d1a4) \
#           + 1/numpy.sqrt(2)*E2*E3 * (a2c3 - a2c4 + c2a3 - c2a4 + a2d3 - a2d4 + d2a3 - d2a4))
    
    bc =  1/numpy.sqrt(2)*(b1c1 + b1d1)
#     bc =    1/numpy.sqrt(2)*(E1**2 * (b1c1 + b1d1) + E2**2 * (b2c2 + b2d2) \
#             + 1/2 * E3**2 *(b3c3 + b4c4 - b3c4 - c3b4 + b3d3 + b4d4 - b3d4 - d3b4) \
#             + E1*E2 * (b1c2 + c1b2 + b1d2 + d1b2) \
#             + 1/numpy.sqrt(2)*E1*E3 * (b1c3 - b1c4 + c1b3 - c1b4 + b1d3 - b1d4 + d1b3 - d1b4) \
#             + 1/numpy.sqrt(2)*E2*E3 * (b2c3 - b2c4 + c2b3 - c2b4 + b2d3 - b2d4 + d2b3 - d2b4))
    
    atrip = 1/2 * (e2a3 - e2a4 - f1a3 + f1a4)
    btrip = 1/2 * (e2b3 - e2b4 - f1b3 + f1b4) 
    ctrip = 1/(2*numpy.sqrt(2)) * (e2c3 + e2d3 - e2c4 - e2d4 - f1c3 - f1d3 + f1c4 + f1d4)
    
    ## 1x1 SINGLET Matrix 1(TT)
    singsing = DIAG + 1/3 * (e2e2 + f1f1 + 1/4*(c3c3 + c4c4 + d3d3 + d4d4) + 2*f1e2 \
             + (e2c3 - e2c4 - e2d3 + e2d4 + f1c3 - f1c4 - f1d3 + f1d4) \
             + 2*(-c3c4 - c3d3 + c3d4 + d3c4 - c4d4 - d3d4))
    
    ## 1x1 QUINTET Matrix 5(TT)
    quintquint = DIAG + 1/6 * (c3c3 + c4c4 + d3d3 + d4d4 + f1f1 + e2e2 + 2*(-c3c4 - c3d3 + c3d4 + e2c3 + f1c3 \
                + d3c4 - c4d4 - e2c4 - f1c4 - d3d4 - e2d3 - f1d3 + e2d4 + f1d4 + f1e2))
    
    ### CONSTRUCTION OF THE FINAL MATRIX for a Triplet environment
    H = [[aa, ab, ac, atrip],[ab, bb, bc, btrip],[ac, bc, cc, ctrip],[atrip, btrip, ctrip, triptrip]]
    return numpy.array(H), singsing, quintquint



The functions below diagonalize the matrices using the scipy package.

In [5]:
def Diagonalize_EnvironSinglet(L,l, coeff_gs, coeff_es, coeff_os, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6):
    """
    This function diagonalizes the 3x3 SINGLET matrix (4e- in 4MOs).
    """
    # Initialization of results
    result = []
    
    # Columns for the DataFrame
    col = ("E(S)1","E(S)2", "E(S)3")
    
    # DEFINE THE TWO MATRIXES (3x3 and 1x1)    
    H, triplet_eigenvalue = Mat_EnvironSinglet(L,l, coeff_gs, coeff_es, coeff_os, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)

    # Obtain the eigenvalues and eigenvectors of H
    eigenvalues, eigenvectors = eigh(H)
    # Extract eigenvalues as real quantities
    eigenvalues = numpy.real(eigenvalues)

    # Data treatment to have in order |g_A -g_A|, |u_A -u_A| and the Singlet OS local states
    new_eigenvectors =[[],[],[]]
    new_eigenvalues = [0,0,0]
    for i in range(3):
        if eigenvectors[i][0]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[0] = eigenvectors[i]
            new_eigenvalues[0] = eigenvalues[i]
            
        if eigenvectors[i][1]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[1] = eigenvectors[i]
            new_eigenvalues[1] = eigenvalues[i]
            
        if eigenvectors[i][2]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[2] = eigenvectors[i]
            new_eigenvalues[2] = eigenvalues[i]
            
    # Create the DataFrame and plug in values    
    result = [[val for val in new_eigenvalues]]
    df = pd.DataFrame(result, columns=col) 
    
    df["E(T)"] = triplet_eigenvalue

    return H, df, new_eigenvectors

In [6]:
def Diagonalize_EnvironTriplet(L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6):
    """
    This function diagonalizes the 4x4 TRIPLET matrix (4e- in 4MOs).
    """
    # Initialization of results
    result = []
    
    # Columns for the DataFrame
    col = ("E(S)1","E(S)2", "E(S)3", "E(T)")
    
    # DEFINE THE TWO MATRIXES    
    H, singlet, quintet = Mat_EnvironTriplet(L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)
        
    # Obtain the eigenvalues and eigenvectors of H
    eigenvalues, eigenvectors = eigh(H)
    # Extract eigenvalues as real quantities
    eigenvalues = numpy.real(eigenvalues)
    
    # Data treatment to have in order |g_A -g_A|, |u_A -u_A|, the singlet OS and the Triplet local states
    new_eigenvectors =[[],[],[],[]]
    new_eigenvalues = [0,0,0,0]
    for i in range(4):
        if eigenvectors[i][0]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[0] = eigenvectors[i]
            new_eigenvalues[0] = eigenvalues[i]
            
        if eigenvectors[i][1]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[1] = eigenvectors[i]
            new_eigenvalues[1] = eigenvalues[i]
            
        if eigenvectors[i][2]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[2] = eigenvectors[i]
            new_eigenvalues[2] = eigenvalues[i]
            
        if eigenvectors[i][3]**2 > 1/2 * sum(j*j for j in eigenvectors[i]):
            new_eigenvectors[3] = eigenvectors[i]
            new_eigenvalues[3] = eigenvalues[i]
        
    # Create the DataFrame and plug in values  
    result = [[val for val in new_eigenvalues]]
    df = pd.DataFrame(result, columns=col)  
    df["E(S)"] = singlet
    df["E(Q)"] = quintet

    return H, df, new_eigenvectors

This code needs information of the AOs integral depending on the system geometry, this was calculated using the Psi4 scripts.

In [7]:
from change_all_test import *

Now let us either do a single calculation or build a Potential Energy Curve.

In [8]:
### This is for a SINGLE calculation for a SINGLET environment.

# Import the integrals for a given geometry
L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6 = dist3_5A_bond0_740A()

# Select the coefficients for the contraction
coeff_gs = 1
coeff_os = 0
coeff_es = 0

H, result_matrix, eigenvectors = Diagonalize_EnvironSinglet(L,l, coeff_gs, coeff_es, coeff_os, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)

print("\033[4mEigenvalues for A Singlet (S) and A Triplet (T)\033[0m")
display(result_matrix)
print("\n\033[4mEigenvectors of A Singlet\033[0m")
print(pd.DataFrame(eigenvectors).to_string(index=False, header=False))

print("\n\033[4mMatrix of A Singlet\033[0m")
print(pd.DataFrame(H).to_string(index=False, header=False))

Eigenvalues for A Singlet (S) and A Triplet (T)


,E(S)1,E(S)2,E(S)3,E(T)
0,-2.253906,-0.633991,-1.285266,-1.647682



Eigenvectors of A Singlet
 0.993643 0.0 0.112578
 0.000000 1.0 0.000000
-0.112578 0.0 0.993643

Matrix of A Singlet
-2.233375  0.181208  0.000000
 0.181208 -0.654521  0.000000
 0.000000  0.000000 -1.285266


In [9]:
### This is for MULTIPLE calculations for a SINGLET environment.

# First initialization of variables
L,l,S,S_i,S_i2,U,t,t_i,t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6 = dist2_0A_bond0_740A()

# The different distances for the Potential Energy Curve
bond0_740A = [dist1_8A_bond0_740A(), dist1_9A_bond0_740A(), dist2_0A_bond0_740A(), dist2_1A_bond0_740A(), dist2_2A_bond0_740A(), dist2_3A_bond0_740A(), 
              dist2_4A_bond0_740A(), dist2_5A_bond0_740A(), dist2_6A_bond0_740A(),dist2_7A_bond0_740A(), dist2_8A_bond0_740A(), dist2_9A_bond0_740A(), 
              dist3_0A_bond0_740A(), dist3_5A_bond0_740A(), dist5_0A_bond0_740A(), dist10_0A_bond0_740A(), dist20_0A_bond0_740A()]

# Global DataFrames
datas = pd.DataFrame(columns= ["E(S)1","E(S)2", "E(S)3", "E(T)"])
eigen = pd.DataFrame()
eigenbis = pd.DataFrame()

# Select the coefficients for the contraction
coeff_gs = 1
coeff_es = 0
coeff_os = 0

# for loop over all the distances
for val in bond0_740A:
    L,l,S,S_i,S_i2,U,t,t_i,t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6 = val
    
    # It is possible to artificially impose the AOs overlap to be zero
    #S = S_i = S_i2 = 0
    
    H, result_matrix, eigenvectors = Diagonalize_EnvironSinglet(L,l, coeff_gs, coeff_es, coeff_os, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)
    datas = datas.append(result_matrix)
    eigen_frame = pd.DataFrame(eigenvectors)
    eigen = eigen.append(eigen_frame)

# One may decide to see the results
display(datas)
display(eigen)
# or to write the output of DataFrames into an OpenOffice file.
#datas.to_excel('test_python_xls',engine='odf')

C:\Users\Pablo\AppData\Local\Temp\ipykernel_25172\1733262402.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  datas = datas.append(result_matrix)
C:\Users\Pablo\AppData\Local\Temp\ipykernel_25172\1733262402.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eigen = eigen.append(eigen_frame)
C:\Users\Pablo\AppData\Local\Temp\ipykernel_25172\1733262402.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  datas = datas.append(result_matrix)
C:\Users\Pablo\AppData\Local\Temp\ipykernel_25172\1733262402.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eigen = eigen.append(eigen_frame)
C:\Users\Pablo\AppData\Local\Temp\ipykernel_2517

,E(S)1,E(S)2,E(S)3,E(T)
0,-1.936737,-0.637614,-1.154387,-1.507256
0,-2.022957,-0.6367,-1.19053,-1.546258
0,-2.087367,-0.635986,-1.217256,-1.575038
0,-2.134972,-0.63545,-1.236861,-1.596102
0,-2.169796,-0.635057,-1.251124,-1.611391
0,-2.195013,-0.634773,-1.261411,-1.622391
0,-2.213092,-0.634568,-1.268766,-1.630234
0,-2.225924,-0.63442,-1.273975,-1.635774
0,-2.234942,-0.634313,-1.277629,-1.63965
0,-2.241217,-0.634234,-1.280169,-1.642335


,0,1,2
0,0.990557,0.0,0.137105
1,0.000000,1.0,0.000000
2,-0.137105,0.0,0.990557
0,0.991593,0.0,0.129393
1,0.000000,1.0,0.000000
2,-0.129393,0.0,0.991593
0,0.992255,0.0,0.124218
1,0.000000,1.0,0.000000
2,-0.124218,0.0,0.992255
0,0.992693,0.0,0.120667


In [10]:
### This is for a SINGLE calculation for a TRIPLET environment.

# Import the integrals for a given geometry
L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6 = dist1_8A_bond0_740A()

H, result_matrix, eigenvectors = Diagonalize_EnvironTriplet(L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2, J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)

print("\033[4mEigenvalues for A Singlet (S) and A Triplet (T)\033[0m")
display(result_matrix)
print("\n\033[4mEigenvectors of A Singlet\033[0m")
display(pd.DataFrame(eigenvectors))
print("\n\033[4mMatrix of A Singlet\033[0m")
display(pd.DataFrame(H))

Eigenvalues for A Singlet (S) and A Triplet (T)


,E(S)1,E(S)2,E(S)3,E(T),E(S),E(Q)
0,-1.528416,-0.006002,-0.613409,-1.06766,-1.276384,-0.890296



Eigenvectors of A Singlet


,0,1,2,3
0,0.993026,0.000000,0.000000,0.117896
1,0.000000,0.993299,-0.115571,0.000000
2,0.000000,0.115571,0.993299,0.000000
3,-0.117896,0.000000,0.000000,0.993026



Matrix of A Singlet


,0,1,2,3
0,-1.507256,0.178234,0.000000,0.000000
1,0.178234,-0.027163,0.000000,0.000000
2,0.000000,0.000000,-0.619476,-0.052147
3,0.000000,0.000000,-0.052147,-1.061593


In [11]:
### This is for MULTIPLE calculations for a TRIPLET environment.

# First initialization of variables
L,l,S,S_i,S_i2,U,t,t_i,t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6 = dist2_0A_bond0_740A()

# The different distances for the Potential Energy Curve
bond0_740A = [dist1_8A_bond0_740A(), dist1_9A_bond0_740A(), dist2_0A_bond0_740A(), dist2_1A_bond0_740A(), dist2_2A_bond0_740A(), dist2_3A_bond0_740A(), 
              dist2_4A_bond0_740A(), dist2_5A_bond0_740A(), dist2_6A_bond0_740A(),dist2_7A_bond0_740A(), dist2_8A_bond0_740A(), dist2_9A_bond0_740A(), 
              dist3_0A_bond0_740A(), dist3_5A_bond0_740A(), dist5_0A_bond0_740A(), dist10_0A_bond0_740A(), dist20_0A_bond0_740A()]

# Global DataFrames
datas = pd.DataFrame(columns= ["E(S)1","E(S)2", "E(S)3", "E(T)", "E(S)","E(Q)"])
eigen = pd.DataFrame()
eigenbis = pd.DataFrame()

# for loop over all the distances
for val in bond0_740A:
    L,l,S,S_i,S_i2,U,t,t_i,t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6 = val
    
    # It is possible to artificially impose the AOs overlap to be zero
    #S = S_i = S_i2 = 0
    
    H, result_matrix, eigenvectors = Diagonalize_EnvironTriplet(L,l, S, S_i, S_i2, U,t, t_i, t_i2,eps,J,K,Jbis,Kbis,omega1,omega2,omega3,theta1,theta2,lambda1,lambda2,J1, gamma1, gamma2, gamma3, gamma4, gamma5, gamma6)
    datas = datas.append(result_matrix)
    eigen_frame = pd.DataFrame(eigenvectors)
    eigen = eigen.append(eigen_frame)

# One may decide to see the results
display(datas)
display(eigen)
# or to write the output of DataFrames into an OpenOffice file.
#datas.to_excel('test_python_xls',engine='odf')

C:\Users\Pablo\AppData\Local\Temp\ipykernel_25172\1158967150.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  datas = datas.append(result_matrix)
C:\Users\Pablo\AppData\Local\Temp\ipykernel_25172\1158967150.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eigen = eigen.append(eigen_frame)
C:\Users\Pablo\AppData\Local\Temp\ipykernel_25172\1158967150.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  datas = datas.append(result_matrix)
C:\Users\Pablo\AppData\Local\Temp\ipykernel_25172\1158967150.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eigen = eigen.append(eigen_frame)
C:\Users\Pablo\AppData\Local\Temp\ipykernel_2517

,E(S)1,E(S)2,E(S)3,E(T),E(S),E(Q)
0,-1.528416,-0.006002,-0.613409,-1.06766,-1.276384,-0.890296
0,-1.567263,-0.018616,-0.638791,-1.065368,-1.318478,-0.939285
0,-1.595921,-0.02745,-0.656892,-1.063855,-1.349168,-0.974899
0,-1.616891,-0.033631,-0.669724,-1.062916,-1.371362,-1.000605
0,-1.632108,-0.037951,-0.678777,-1.062359,-1.387282,-1.019027
0,-1.643054,-0.040965,-0.685137,-1.062044,-1.398609,-1.032131
0,-1.650856,-0.043064,-0.689586,-1.061873,-1.406603,-1.041379
0,-1.656366,-0.044524,-0.692683,-1.061786,-1.412198,-1.047854
0,-1.66022,-0.045536,-0.694828,-1.061746,-1.41608,-1.05235
0,-1.66289,-0.046238,-0.696304,-1.061731,-1.418751,-1.055446


,0,1,2,3
0,0.993026,0.000000,0.000000,0.117896
1,0.000000,0.993299,-0.115571,0.000000
2,0.000000,0.115571,0.993299,0.000000
3,-0.117896,0.000000,0.000000,0.993026
0,0.993195,0.000000,0.000000,0.116463
...,...,...,...,...
3,-0.112544,0.000000,0.000000,0.993647
0,-0.993647,0.000000,0.000000,0.112544
1,0.000000,1.000000,0.000000,0.000000
2,0.000000,0.000000,1.000000,0.000000
